In [1]:
import itertools, re
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

In [2]:
Logging.setup_logging()

<Logger PySpice (INFO)>

In [3]:
def empty_ir(title="Candidate"):
    return {"title": title, "nodes": ["in","out","0"], "components": []}

In [ ]:
def add_vin_and_load(ir, ac=1.0, Rload=10e3):
    # AC=1 V small-signal source recorded in IR
    ir["components"].append({"kind":"VAC","name":"in", "pos":"in","neg":"0","ac": ac})
    # Tiny "wire" placeholder for in->out path (a rewire anchor for series insertions)
    ir["components"].append({"kind":"R","name":"Rpath","n1":"in","n2":"out","value":1e-3})
    # Output load
    ir["components"].append({"kind":"R","name":"Rload","n1":"out","n2":"0","value": Rload})

In [5]:
from PySpice.Spice.Netlist import SubCircuitFactory

class RC_LowPass(SubCircuitFactory):
    __name__  = 'rc_lp'
    __nodes__ = ('in', 'out', '0')
    def __init__(self, R=1@u_kΩ, C=1@u_uF):
        super().__init__()
        self.R(1, 'in', 'out', R)
        self.C(1, 'out', '0',  C)

In [6]:
SUBCKT_REGISTRY = {
    'rc_lp':  RC_LowPass,
}

In [7]:
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

def emit_pyspice(ir: dict, subckt_registry=None) -> Circuit:
    """
    Convert an IR dictionary into a PySpice Circuit.
    Supports R, C, L, VAC, SUBCKT.
    SUBCKT expects:
      comp = {
        "kind":"SUBCKT", "name":"U1", "model":"rc_tee",
        "ports": {"in":"n5","out":"n6","ref":"n7"},   # formal->actual
        "params": {"R1": "1k", "R2": "2k", "C1": "10n"}  # optional
      }
    """
    if subckt_registry is None:
        subckt_registry = {}

    def _resolve_node(n, circuit):
        # Allow either '0' or 'gnd' to mean ground
        if isinstance(n, str) and n.strip().lower() in ('0', 'gnd'):
            return circuit.gnd
        return n

    c = Circuit(ir["title"])
    # Track which subckt models we've registered on this Circuit
    c._registered_subckts = set()

    for comp in ir["components"]:
        k = comp["kind"].upper()

        if k == "R":
            c.R(comp["name"], _resolve_node(comp["n1"], c), _resolve_node(comp["n2"], c),
                float(comp["value"]) @ u_Ohm)

        elif k == "C":
            c.C(comp["name"], _resolve_node(comp["n1"], c), _resolve_node(comp["n2"], c),
                float(comp["value"]) @ u_F)

        elif k == "L":
            c.L(comp["name"], _resolve_node(comp["n1"], c), _resolve_node(comp["n2"], c),
                float(comp["value"]) @ u_H)

        elif k == "VAC":
            ac_amp = float(comp["ac"])
            pos = _resolve_node(comp["pos"], c)
            neg = _resolve_node(comp["neg"], c)
            if hasattr(c, "AcVoltageSource"):
                c.AcVoltageSource(comp["name"], pos, neg, amplitude=ac_amp @ u_V)
            else:
                c.V(comp["name"], pos, neg, f"dc 0 ac {ac_amp}")

        elif k == "SUBCKT":
            model = comp["model"]
            ports = comp["ports"]           # dict: formal -> actual node name
            params = comp.get("params", {}) # dict of parameter strings or Unit values

            SubcktClass = subckt_registry[model]

            # Register the subcircuit ONCE on this Circuit
            if model not in c._registered_subckts:
                c.subcircuit(SubcktClass(**{k:(v) for k,v in params.items()}))
                c._registered_subckts.add(model)

            # Build node list in the formal port order declared by the subckt class
            formal_order = getattr(SubcktClass, '__nodes__', tuple(ports.keys()))
            node_list = [_resolve_node(ports[p], c) for p in formal_order]

            # Instantiate X device; model name must match class.__name__
            c.X(comp["name"], SubcktClass.__name__, *node_list)

        else:
            raise ValueError(f"Unsupported kind: {k}")

    return c


In [ ]:
def add_series(ir, name, kind, value):
    """
    Insert a series element along the in->out path by progressively
    extending the chain of intermediate nodes.
    """
    # Make a unique new node name
    next_idx = sum(1 for c in ir["components"] if c["kind"] in ("R","C","L"))
    new_node = f"n{next_idx}"

    # Find the last element feeding 'out' and redirect it to new_node
    for comp in ir["components"]:
        if comp.get("n2") == "out":   # safe lookup
            comp["n2"] = new_node
            break

    # Append the new element from new_node to out
    ir["components"].append({
        "kind": kind,
        "name": name,
        "n1": new_node,
        "n2": "out",
        "value": value
    })